<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Word2Vec-Skipgram-Keras</H1></u></center>

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
import gensim

C:\Anaconda36\envs\tf111_gpu\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
remove_terms = punctuation + '0123456789'

In [5]:
def preprocessing(text):
    words = word_tokenize(text)
    tokens = [w for w in words if w.lower() not in remove_terms]
    #stopw = stopwords.words('english')
    #tokens = [token for token in tokens if token not in stopw]
    # remove words less than three letters
    #tokens = [word for word in tokens if len(word)>=3]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [6]:
#path = get_file('cosmos.txt', origin="https://www.gutenberg.org/files/8172/8172-0.txt")

In [7]:
corpus = open("../data/History_of_Astronomy.txt", encoding="utf8").readlines()

In [8]:
corpus = [preprocessing(sentence) for sentence in corpus if sentence.strip() !='']
corpus

['Project Gutenberg EBook of History of Astronomy by George Forbes',
 'Title History of Astronomy',
 'Author George Forbes',
 'Illustration SIR ISAAC NEWTON From the bust by Roubiliac In Trinity',
 'College Cambridge',
 'HISTORY OF ASTRONOMY',
 'BY',
 'GEORGE FORBES',
 'INST',
 'FORMERLY PROFESSOR OF NATURAL PHILOSOPHY ANDERSON COLLEGE GLASGOW',
 'AUTHOR OF THE TRANSIT OF VENUS RENDU THEORY OF THE GLACIERS OF',
 'SAVOY ETC',
 'CONTENTS',
 'PREFACE',
 'BOOK I THE GEOMETRICAL PERIOD',
 'PRIMITIVE ASTRONOMY AND ASTROLOGY',
 'ANCIENT ASTRONOMY CHINESE AND CHALDÆANS',
 'ANCIENT GREEK ASTRONOMY',
 'THE REIGN OF EPICYCLES FROM PTOLEMY TO COPERNICUS',
 'BOOK II THE DYNAMICAL PERIOD',
 'DISCOVERY OF THE TRUE SOLAR SYSTEM TYCHO BRAHE KEPLER',
 'GALILEO AND THE TELESCOPE NOTIONS OF GRAVITY BY HORROCKS ETC',
 'SIR ISAAC NEWTON LAW OF UNIVERSAL GRAVITATION',
 'NEWTON SUCCESSORS HALLEY EULER LAGRANGE LAPLACE ETC',
 'DISCOVERY OF NEW PLANETS HERSCHEL PIAZZI ADAMS AND LE',
 'VERRIER',
 'BOOK III OBSER

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [10]:
X_train_tokens = tokenizer.texts_to_sequences(corpus)

In [11]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

4411

In [12]:
items = tokenizer.word_index.items()

## Creating the Skipgram Model:

In [13]:
dim_embedddings = 300

# inputs
inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(vocab_size, dim_embedddings)(inputs)

# context
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(vocab_size, dim_embedddings)(c_inputs)

d = Dot(axes=2)([w, c])

d = Reshape((1,), input_shape=(1, 1))(d)
d = Activation('sigmoid')(d)

model = Model(inputs=[inputs, c_inputs], outputs=d)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 300)       1323300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 300)       1323300     input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam')

## Training the model:

In [15]:
%%time
n_epochs = 15
for epoch in range(n_epochs):
    loss = 0.
    for i, doc in enumerate(X_train_tokens):
        data, labels = skipgrams(sequence=doc, vocabulary_size=vocab_size, window_size=4)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if x:
            loss += model.train_on_batch(x, y)

    print('Epoch:', epoch, '\tLoss:', loss)

Epoch: 0 	Loss: 1855.122561082244
Epoch: 1 	Loss: 1490.9883154407144
Epoch: 2 	Loss: 1388.3679221607745
Epoch: 3 	Loss: 1270.372354991734
Epoch: 4 	Loss: 1164.389208954759
Epoch: 5 	Loss: 1063.671567552723
Epoch: 6 	Loss: 982.9436835721135
Epoch: 7 	Loss: 913.5112223466858
Epoch: 8 	Loss: 857.7043578715529
Epoch: 9 	Loss: 809.6147848761175
Epoch: 10 	Loss: 771.9398003910901
Epoch: 11 	Loss: 743.7286821631715
Epoch: 12 	Loss: 722.955197765812
Epoch: 13 	Loss: 703.9326986923697
Epoch: 14 	Loss: 685.0070044388121
Wall time: 4min 59s


## Saving the model:

In [18]:
f = open('../data/word2vec-skipgrams1.txt' ,'w', encoding="utf8")
f.write('{} {}\n'.format(vocab_size-1, dim_embedddings))

weights = model.get_weights()[0]
for word, i in items:
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(weights[i, :])))))
f.close()

## Loading the model:

In [19]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('../data/word2vec-skipgrams1.txt', binary=False)

In [20]:
w2v.most_similar(positive=['solar'])

C:\Anaconda36\envs\tf111_gpu\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gaseous', 0.4597316086292267),
 ('researches', 0.4340711534023285),
 ('harmful', 0.4325682818889618),
 ('physics', 0.4134013056755066),
 ('system', 0.41045206785202026),
 ('news', 0.39892005920410156),
 ('compressed', 0.3597968816757202),
 ('nautical', 0.35958218574523926),
 ('substitute', 0.3580801486968994),
 ('toledo', 0.3563697040081024)]

In [21]:
w2v.most_similar(positive=['system'])

[('advocated', 0.4853593111038208),
 ('solar', 0.4104520082473755),
 ('main', 0.40048810839653015),
 ('relationship', 0.3770222067832947),
 ('answered', 0.3546488583087921),
 ('golden', 0.33952847123146057),
 ('clue', 0.33863192796707153),
 ('harmful', 0.33825457096099854),
 ('driving', 0.3371925354003906),
 ('predominance', 0.33416372537612915)]

In [22]:
w2v.most_similar(positive=['kepler'])

[('wanderer', 0.5605933666229248),
 ('johannes', 0.5162597894668579),
 ('reitlinger', 0.5112121105194092),
 ('independence', 0.4941261410713196),
 ('strassburg', 0.4905438721179962),
 ('yielding', 0.4500398635864258),
 ('slavish', 0.44416677951812744),
 ('contemplated', 0.44190627336502075),
 ('abolish', 0.4218079745769501),
 ('johann', 0.4171288013458252)]

## References:

https://keras.io/preprocessing/sequence/

https://www.tensorflow.org/tutorials/representation/word2vec